# Searching or questioning all the sources
In this notebook, we create a chain that can ask all the different sources a question. Using functions we can choose the right source. We use the language model to make the selection.
- Product search: use the VectorStore retriever making use of Amazon OpenSearch Service
- Question: use the VectorStore QA retriever to use content from the vector store and OpenAI to write an asnwer
- Find Opening times: Use a normal query against Amazon OpenSearch Service

## Initialise the OpenSearch connection 
Notice that we do not provide a default alias. We need to provide the name of the index or alias on each request.

In [2]:
from retriever import find_auth_opensearch, OpenSearchClient

config = find_auth_opensearch()
client = OpenSearchClient(config)

if client.ping():
    print("We have a connection to the Amazon OpenSearch Cluster")
else:
    print("ERROR: no connection to the Amazon OpenSearch Cluster")

We have a connection to the Amazon OpenSearch Cluster


## Initialise the product search

In [14]:
import os

from langchain.vectorstores import OpenSearchVectorSearch
from langchain.embeddings import OpenAIEmbeddings
from opensearchpy import RequestsHttpConnection
from dotenv import load_dotenv

load_dotenv()

vector_store_products = OpenSearchVectorSearch(
    index_name="sg-products",
    embedding_function=OpenAIEmbeddings(openai_api_key=os.getenv('OPEN_AI_API_KEY')),
    opensearch_url=f"https://{config['host']}:{config['port']}",
    use_ssl=True,
    verify_certs=True,
    http_auth=config["auth"],
    connection_class=RequestsHttpConnection
)

def execute_product_search(query: str):
    found_docs = vector_store_products.similarity_search_with_score(query=query, text_field="title", vector_field="title_vector")
    results = []
    for doc, _score in found_docs:
        results.append({"title": doc.page_content, "score": _score})

    return results

In [4]:
print(execute_product_search(query="The lord of the rings"))

[{'title': 'Frodo™ & Gollem™', 'score': 0.7835052}, {'title': 'Aragorn™ & Arwen™', 'score': 0.7829712}, {'title': 'Gandalf de Grijze™ & Balrog™', 'score': 0.75238377}, {'title': 'Slag om Endor™ helden', 'score': 0.7370234}]


## Initialise the content search

In [15]:
vector_store = OpenSearchVectorSearch(
    index_name="sg-content",
    embedding_function=OpenAIEmbeddings(openai_api_key=os.getenv('OPEN_AI_API_KEY')),
    opensearch_url=f"https://{config['host']}:{config['port']}",
    use_ssl=True,
    verify_certs=True,
    http_auth=config["auth"],
    connection_class=RequestsHttpConnection
)

from langchain.chains import RetrievalQA
from langchain import PromptTemplate, OpenAI

prompt_template = """Use the context to answer the question. If you don't know the 
    answer, just say that you don't know, don't make up an answer.

    {context}

    Question: {question}:"""

custom_prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

chain_type_kwargs = {"prompt": custom_prompt}
chain = RetrievalQA.from_chain_type(
    llm=OpenAI(openai_api_key=os.getenv('OPEN_AI_API_KEY')),
    chain_type="stuff",
    retriever=vector_store.as_retriever(),
    chain_type_kwargs=chain_type_kwargs
)

def execute_content_search(query: str):
    return chain.run(query)

In [7]:
print(execute_content_search("What is your return policy"))



Our return policy can be found on the Returns & Refunds page. You can access this page from the bottom of the home page.


In [19]:
from langchain.chains.openai_functions import create_openai_fn_chain
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI


def run_conversation(message_content: str):
    llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key=os.getenv('OPEN_AI_API_KEY'))

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system", "You are a service with access to multiple data sources. Each data source is represented by a different function."),
            ("human", "Make calls to the relevant function to answer the provided question or search for a product. Always provided the exact query as provided by the user to the function: {input}"),
        ]
    )

    functions = [execute_content_search, execute_product_search]
    function_chain = create_openai_fn_chain(
        functions=functions,
        llm=llm,
        prompt=prompt,
        verbose=True)

    return function_chain.run(message_content)


In [20]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool


def run_agent(message_content: str):
    llm = ChatOpenAI(model="gpt-4", temperature=0, openai_api_key=os.getenv('OPEN_AI_API_KEY'))

    tools = [
        Tool(name="ExecuteContentSearch",
             func=execute_content_search,
             description="Useful for obtaining answers to questions about help for the website, your account and sustainability."
             ),
        Tool(name="ExecuteProductSearch",
             func=execute_product_search,
             description="Useful for finding products related to the search terms that are provided."
             ),
    ]

    agent = initialize_agent(tools=tools, llm=llm, agent=AgentType.OPENAI_FUNCTIONS, verbose=True)
    return agent.run(message_content)


In [22]:
# message = "lord of the rings"
message = "Do you use reusable packaging"

response = run_agent(message_content=message)

print(response)



> Entering new AgentExecutor chain...

Invoking: `ExecuteContentSearch` with `reusable packaging`


 What is [Your Company Name]'s commitment to reusable packaging?

[Your Company Name] is committed to reducing single-use plastics and opting for biodegradable, compostable, or recyclable packaging materials. We prioritize items that align with our commitment to a greener future, focusing on eco-friendly materials and products designed for longevity and reusability.Our company is committed to reducing single-use plastics and opting for biodegradable, compostable, or recyclable packaging materials. We prioritize items that align with our commitment to a greener future, focusing on eco-friendly materials and products designed for longevity and reusability.

> Finished chain.
Our company is committed to reducing single-use plastics and opting for biodegradable, compostable, or recyclable packaging materials. We prioritize items that align with our commitment to a greener future, focusing 